<a href="https://colab.research.google.com/github/albertoliuzzo/progetti_AI/blob/main/Machine_learning_base_modello_di_previsione_per_il_mercato_immobiliare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet

df=pd.read_csv("housing.csv") #carico il dataset

#DATA PREPROCESSING

df.isna().sum()  #verifico la presenza di valori nulli

corr_matrix = df.corr() # calcolo la matrice di correlazione e la visualizzo, per selezione delle feature (eliminare feature non rilevanti o altamente correlate tra loro)
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Matrice di Correlazione tra le Feature")
plt.show()

# verifica ed eliminazione outliers
Q1 = df["price"].quantile(0.20) #metodo IQR
Q3 = df["price"].quantile(0.80)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

new_df = df[(df["price"] < upper_bound)] #nuovo dataset senza outliers

X=new_df.drop(["price","hotwaterheating","furnishingstatus","basement","guestroom","mainroad"], axis=1) #creo il dataset X senza il target e le feature non rilevanti
y=new_df["price"] #creo il target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) #divido in dataset di training e dataset di test

ct = ColumnTransformer(  #creo l'istanza che standardizza le variabili continue e ordinali (escludendo le binarie)
    transformers=[('scaler', StandardScaler(), ["area","bedrooms", "bathrooms", "stories", "parking"] ) ],
    remainder = "passthrough")

X_train = ct.fit_transform(X_train)  #fitto e trasformo il modello ottenendo i dataset con le feature standardizzate
X_test = ct.transform(X_test)

#CREAZIONE DEI MODELLI

param_grid = {"alpha": np.logspace(-4, 1, 30)} #creo 30 valori del parametro alpha che vanno da 10^(-4) a 10^1

#utilizzo la GridSearchCV per trovare i migliori valori di alpha che ottimizzino il coefficiente di determinazione l'MSE
grid_search_R = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_R.fit(X_train, y_train)
print("Miglior valore di alpha per Ridge:", grid_search_R.best_params_['alpha'])

grid_search_L = GridSearchCV(Lasso(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_L.fit(X_train, y_train)
print("Miglior valore di alpha per Lasso:", grid_search_L.best_params_['alpha'])

grid_search_El = GridSearchCV(ElasticNet(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_El.fit(X_train, y_train)
print("Miglior valore di alpha per ElasticNet:", grid_search_El.best_params_['alpha'])

#creo i modelli di Regression Ridge, Lasso ed ElasticNet (con gli alpha ottenuti)
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

model_R = Ridge(alpha=3.039195382313201)
model_R.fit(X_train, y_train)

model_L = Lasso(alpha=0.0001)
model_L.fit(X_train, y_train)

model_E = ElasticNet(alpha=0.01743328822199989)
model_E.fit(X_train, y_train)

#VALUTAZIONE DEI MODELLI

#valuto i 3 modelli sul set di training tramite cross validation
score_R = cross_val_score(model_R, X_train, y_train, cv=5, scoring="r2").mean()  #calcolo il coefficiente di determinazione sul set di training, come media degli R2 dei vari folds
score_L = cross_val_score(model_L, X_train, y_train, cv=5, scoring="r2").mean()
score_E = cross_val_score(model_E, X_train, y_train, cv=5, scoring="r2").mean()

score_R_test = model_R.score(X_test, y_test)  #calcolo il coefficiente di determinazione sul set di test, grazie alla funzione score()
score_L_test = model_L.score(X_test, y_test)
score_E_test = model_E.score(X_test, y_test)

print(f"Ridge Regression - Train R2: {score_R:.3f}, Test R2: {score_R_test:.3f}") #stampa dei risultati
print(f"Lasso Regression - Train R2: {score_L:.3f}, Test R2: {score_L_test:.3f}")
print(f"Elastic Net - Train R2: {score_E:.3f}, Test R2: {score_E_test:.3f}")
#Ridge Regression - Train R2: 0.625, Test R2: 0.618
#Lasso Regression - Train R2: 0.625, Test R2: 0.620
#Elastic Net - Train R2: 0.625, Test R2: 0.617

#MSE
y_train_pred_R = model_R.predict(X_train)  #calcolo le predizioni dei modelli rispetto al set di training
y_train_pred_L = model_L.predict(X_train)
y_train_pred_E = model_E.predict(X_train)

y_test_pred_R = model_R.predict(X_test)  #calcolo le predizioni dei modelli rispetto al set di test
y_test_pred_L = model_L.predict(X_test)
y_test_pred_E = model_E.predict(X_test)

mse_train_R = mean_squared_error(y_train, y_train_pred_R)  #calcolo l'mse sul set di training
mse_train_L = mean_squared_error(y_train, y_train_pred_L)
mse_train_E = mean_squared_error(y_train, y_train_pred_E)

mse_test_R = mean_squared_error(y_test, y_test_pred_R) #calcolo l'mse sul set di test
mse_test_L = mean_squared_error(y_test, y_test_pred_L)
mse_test_E = mean_squared_error(y_test, y_test_pred_E)

print(f"Ridge Regression - Train MSE: {mse_train_R:.2f}, Test MSE: {mse_test_R:.2f}") #stampa dei risultati
print(f"Lasso Regression - Train MSE: {mse_train_L:.2f}, Test MSE: {mse_test_L:.2f}")
print(f"Elastic Net - Train MSE: {mse_train_E:.2f}, Test MSE: {mse_test_E:.2f}")
#Ridge Regression - Train MSE: 1189968293814.19, Test MSE: 1605549997423.95
#Lasso Regression - Train MSE: 1189713915417.60, Test MSE: 1597626583200.85
#Elastic Net - Train MSE: 1190105262926.58, Test MSE: 1607541751610.49

#calcolo dei coefficienti non nulli
non_zero_ridge = np.count_nonzero(model_R.coef_)
non_zero_lasso = np.count_nonzero(model_L.coef_)
non_zero_elastic = np.count_nonzero(model_E.coef_)

print(f"Numero di coefficienti non nulli:") #stampa dei risultati
print(f"Ridge Regression: {non_zero_ridge}")
print(f"Lasso Regression: {non_zero_lasso}")
print(f"Elastic Net: {non_zero_elastic}")

#Numero di coefficienti non nulli:
#Ridge Regression: 7
#Lasso Regression: 7
#Elastic Net: 7

#confronto dei 3 modelli con grafico a barre
mse_results = pd.DataFrame({   # creo un dataframe con i valori degli MSE per il confronto tra i modelli
    "Modelli": ["Ridge", "Lasso", "ElasticNet"],
    "MSE Train": [mse_train_R, mse_train_L, mse_train_E],
    "MSE Test": [mse_test_R, mse_test_L, mse_test_E]
})

# visualizzazione degli MSE con un grafico a barre
plt.figure(figsize=(10,6))
mse_results.set_index("Modelli").plot(kind="bar", figsize=(10,6), alpha=0.8)
plt.title("Confronto degli MSE tra i modelli")
plt.ylabel("Mean Squared Error")
plt.xticks(rotation=0)
plt.grid()
plt.show()

#Visualizzazione della distribuzione dei residui per valutare l'adeguatezza del modello.

# calcolo dei residui (differenza tra valori reali e predetti)
residuals_R = y_test - y_test_pred_R
residuals_L = y_test - y_test_pred_L
residuals_E = y_test - y_test_pred_E

# visualizzazione della distribuzione dei residui per ciascun modello
plt.figure(figsize=(15,5))

plt.subplot(1, 3, 1)
sns.histplot(residuals_R, bins=50, kde=True, color="blue")
plt.title("Distribuzione dei residui - Ridge")

plt.subplot(1, 3, 2)
sns.histplot(residuals_L, bins=40, kde=True, color="red")
plt.title("Distribuzione dei residui - Lasso")

plt.subplot(1, 3, 3)
sns.histplot(residuals_E, bins=50, kde=True, color="green")
plt.title("Distribuzione dei residui - Elastic Net")

plt.show()


# Visualizzazione dell'andamento dei coefficienti dei modelli rispetto ai parametri di regolarizzazione

alphas = np.logspace(-4, 4, 30)  # creo 30 valori per alpha da 0.0001 a 10000

# liste per memorizzare i coefficienti
ridge_coeff = []
lasso_coeff = []
elastic_coeff = []

# ciclo for per testare diversi alpha
for alpha in alphas:
    model_R = Ridge(alpha=alpha)
    model_L = Lasso(alpha=alpha)
    model_E = ElasticNet(alpha=alpha)

    model_R.fit(X_train, y_train)
    model_L.fit(X_train, y_train)
    model_E.fit(X_train, y_train)

    ridge_coeff.append(model_R.coef_)
    lasso_coeff.append(model_L.coef_)
    elastic_coeff.append(model_E.coef_)

# conversione in array numpy
ridge_coeff = np.array(ridge_coefs)
lasso_coeff = np.array(lasso_coefs)
elastic_coeff = np.array(elastic_coefs)

# grafico per visualizzare l'andamento dei coefficienti in Ridge
plt.figure(figsize=(10,6))
for i in range(ridge_coeff.shape[1]):
    plt.plot(alphas, ridge_coeff[:, i], label=f'Feature {i+1}')
plt.xscale("log")
plt.xlabel("Alpha (Lambda)")
plt.ylabel("Valore dei Coefficienti")
plt.title("Andamento dei coefficienti in Ridge Regression")
plt.legend()
plt.show()

# grafico per Lasso
plt.figure(figsize=(10,6))
for i in range(lasso_coeff.shape[1]):
    plt.plot(alphas, lasso_coeff[:, i], label=f'Feature {i+1}')
plt.xscale("log")
plt.xlabel("Alpha (Lambda)")
plt.ylabel("Valore dei Coefficienti")
plt.title("Andamento dei coefficienti in Lasso Regression")
plt.legend(loc="best", bbox_to_anchor=(1,1))
plt.show()

# grafico per ElasticNet
plt.figure(figsize=(10,6))
for i in range(elastic_coeff.shape[1]):
    plt.plot(alphas, elastic_coeff[:, i], label=f'Feature {i+1}')
plt.xscale("log")
plt.xlabel("Alpha (Lambda)")
plt.ylabel("Valore dei Coefficienti")
plt.title("Andamento dei coefficienti in Elastic Net")
plt.legend()
plt.show()
